![alt text](path_to_banner_image.jpg)
# Trends in Supplemental Education Program
Author: Elijah Lopez

___ 
## Overview

This analysis outlines actionable strategies to improve lead conversion, customer retention, and program effectiveness for a supplemental education program. To optimize lead conversion, this business should re-engage open leads and inactive accounts, highlight long-term value, and prioritize enrollment efforts during high-interest periods like October. Tailored marketing strategies targeting parents of late-elementary school students in key zip codes combined with seasonal promotions during the summer can further enhance lead generation. Referral programs and upselling private sessions to high schoolers present additional opportunities to boost revenue and engagement.

Retention and program effectiveness can be strengthened through active communication with families, clear progress tracking, and incentives for long-term commitments. Monitoring student performance at critical milestones ensures mastery and alignment with family goals, while time-based metrics provide clarity on program expectations and outcomes. By aligning these efforts, this business can foster trust, improve operational efficiency, and deliver meaningful results for both students and families.

___
## Business Problem

This supplemental education program seeks to improve its ability to convert prospective leads into active enrollments, retain students for longer periods, and deliver measurable outcomes that align with family expectations. Addressing these challenges requires identifying patterns in lead behavior, tailoring marketing efforts to key demographics, and ensuring that program offerings effectively meet the diverse needs of students.

___
## Contingencies

### Sources

In [ ]:
# Leads
leads_location = 'data/location_1/leads.csv'

# Accounts
accounts_location = 'data/location_1/accounts.csv'

# Assessment Progress
assessment_progress_location = 'data/location_1/assessment_progress.csv'

### Imports

In [ ]:
# Standard Data Science Toolkit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.style.use("ggplot")
%matplotlib inline
import seaborn as sns

# Inferential Statistical Tests
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

### Helper Functions

In [ ]:
def standardize_columns(df):
    """
    Makes column names lower case and replaces spaces with underscores.

    Parameters:
        df (pd.dfFrame) - The data containing the groups and values.

    Returns:
        df
    """
    
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    return df

In [ ]:
def plot_categorical(df):
    """
    Identifies categorical columns ('object' datatypes) and plots as horizontal bar chart.

    Parameters:
        df (pd.dfFrame) - The data containing the groups and values.

    Returns:
        Plot    
    """

    # Isolate categorical columns
    cat_cols = []
    for column in df.columns:
        if df.dtypes[column] == 'O':
            cat_cols.append(column)

    # Calculate the number of rows and columns needed, limiting to 3 graphs per row
    num_cols_per_row = 3
    num_rows = int(np.ceil(len(cat_cols) / num_cols_per_row))

    # Create subplots with the dynamic grid
    fig, axes = plt.subplots(num_rows, num_cols_per_row, figsize=(15, 5))
    axes = axes.flatten()  # Flatten the grid for easy indexing

    # Plot each categorical column
    for i, column in enumerate(cat_cols):
        x = df[column].value_counts().index.astype(str)
        y = df[column].value_counts().values
        
        # Barh plot for the column
        axes[i].barh(x, y, color='skyblue')
        axes[i].set_title(column)
        axes[i].set_xlabel('Count')

    # Adjust layout to prevent overlap
    return plt.tight_layout();

In [ ]:
def plot_numeric(df):
    """
    Identifies numeric columns ('float64' & 'int64' datatypes) and plots as horizontal bar chart.

    Parameters:
        df (pd.dfFrame) - The data containing the groups and values.

    Returns:
        Plot    
    """

    # Isolate numeric columns
    num_cols = []
    for column in df.columns:
        if df.dtypes[column] == 'float64' or df.dtypes[column] == 'int64':
            num_cols.append(column)

    # Calculate the number of rows needed, limiting 3 graphs per row
    num_cols_per_row = 3
    num_rows = int(np.ceil(len(num_cols) / num_cols_per_row))

    fig, axes = plt.subplots(num_rows, num_cols_per_row, figsize=(12, 6))
    axes = axes.flatten()

    # Apply to each column
    for i, column in enumerate(num_cols):
        # Plot a histogram for each numeric column
        axes[i].hist(df[column], color='skyblue', edgecolor='black', alpha=0.7)
        
        # Add title and labels
        axes[i].set_title(f'{column}')
        # axes[i].set_xlabel(column)
        axes[i].set_ylabel('Frequency')

    # Adjust layout to prevent overlap
    return plt.tight_layout();

In [ ]:
def plot_x_over_time(x, y, title, y_label, line_label, average_label_color='#E2231A'):
    """
    Plots a time series with a line representing the average value.

    Parameters:
        x (var): Data for x-axis.
        y (var): Data for y-axis.
        title (str): The chart title.
        y_label (str): Label for the y-axis.
        line_label (str): Label for the line in the legend.
        average_label_color (str): Color for the average line (default: '#E2231A').

    Returns:
        Plot
    """

    plt.figure(figsize=(12, 6))

    # Plot the data
    plt.plot(x, y, marker='o', linestyle='-', color='skyblue', label=line_label)

    # Calculate and plot the mean
    average_value = y.mean()
    plt.axhline(y=average_value, color=average_label_color, linestyle='--', label=f'Average ({average_value:.0f})')

    # Add title, labels, and legend
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel(y_label)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show();

In [ ]:
def plot_x_per_month(y, title, y_label, bar_color='skyblue', avg_line_color='#E2231A'):
    """
    Plots a bar chart of monthly df with a horizontal line showing the average.

    Parameters:
        y (var): Data for y-axis, indexed by three letter abbreviation for month.
        average (float): The value of the horizontal average line.
        title (str): The chart title.
        y_label (str): Label for the y-axis.
        bar_color (str): The color of the bars (default 'skyblue').
        avg_line_color (str): The color of the average line (default '#E2231A').

    Returns:
        Plot
    """
    
    # Sort by the month order
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    y = y.reindex(month_order)
    
    # Plot the bar chart
    plt.figure(figsize=(10, 6))
    y.plot(kind='bar', color=bar_color, label='')
    
    # Add a horizontal average line
    average = y.mean()
    plt.axhline(y=average, color=avg_line_color, linestyle='--', label=f'Average ({average:.0f})')
    
    # Add title and labels
    plt.title(title)
    plt.xlabel('Month')
    plt.ylabel(y_label)
    plt.xticks(rotation=0)
    plt.legend()
    plt.tight_layout()
    plt.show();

In [ ]:
def perform_anova(df, group_col, value_col, alpha):
    """
    Performs a one-way ANOVA test to compare means across groups.
    
    Parameters:
        df (pd.dfFrame) - The data containing the groups and values.
        group_col (str) - The column name representing the groups
        value_col (str) - The column name representing the values to compare
        alpha (float) - Confidence level
    
    Returns:
        dict: A dictionary containing the F-Statistic, P-Value, and interpretation
    """
    
    # Group the df by the specified group column
    grouped = df.groupby(group_col)[value_col]
    
    # Extract values for each group into separate lists
    grouped_values = [grouped.get_group(group).values for group in grouped.groups.keys()]
    
    # Perform ANOVA test
    f_stat, p_value = f_oneway(*grouped_values)
    
    # Interpretation
    result = {
        "F-Statistic": f_stat,
        "P-Value": p_value,
        "Significance": "Reject the null in favor of the alternative hypothesis."
        if p_value < 0.05 else "Fail to reject the null hypothesis."
    }
    
    return result

In [ ]:
def perform_tukeys_hsd(df, group_col, value_col, alpha=0.05):
    """
    Perform Tukey's HSD test to determine significant differences between groups 
    and return results as a DataFrame with significant differences isolated.

    Parameters:
        df (pd.DataFrame): The data containing the groups and values.
        group_col (str): The column representing groups (e.g., 'month').
        value_col (str): The column representing the dependent variable (e.g., 'lead_count').
        alpha (float): Significance level for the test (default is 0.05).

    Returns:
        tukey_df (pd.DataFrame): A DataFrame containing Tukey HSD results.
        significant_results (pd.DataFrame): Rows with statistically significant differences.
    """
    # Perform Tukey's HSD test
    tukey = pairwise_tukeyhsd(
        endog=df[value_col],  # The dependent variable
        groups=df[group_col],  # The grouping variable
        alpha=alpha  # Significance level
    )
    
    # Print the summary
    print(tukey)

    # Extract Tukey's HSD results into a DataFrame
    tukey_df = pd.DataFrame(
        data=tukey.summary().data[1:],  # Extract data excluding the header row
        columns=tukey.summary().data[0]  # Extract the column names
    )

    # Isolate statistically significant results
    significant_results = tukey_df.loc[tukey_df['reject'] == True]

    return significant_results

In [ ]:
def assign_grade_range(grade):
    """
    Assign grade range based on grade

    Parameters:
        grade from df[col]

    Returns:
        grade range ('Elementary', 'Middle', or 'High')
    """

    if grade >= 9:
        return 'High'
    elif 5 < grade < 9:
        return 'Middle'
    else:
        return 'Elementary'

In [ ]:
def plot_histogram(data, title, x_label, mean_median='median', x_ticks_interval=1, bins=None):
    """
    Plots a histogram of a df column with a vertical line showing the median (default) or average.

    Parameters:
        data (var): Column from data frame to be plotted.
        title (str): The chart title.
        x_label (str): Label for the x-axis.
        mean_median (str): Specify whether to plot the vertical line as the mean or median value.

    Returns:
        Plot
    """    
    
    # Data to plot
    data = data
    # median = data.median()
    x_ticks = range(int(data.min()), int(data.max()) + 1, x_ticks_interval)
    
    if mean_median == 'median':
        vline = data.median()
    elif mean_median == 'mean':
        vline = data.mean()
    else:
        vline = None
    
    if bins is None:
        bins = int(data.max() - data.min())
    
    # Plot
    plt.hist(data, bins=bins, color='skyblue')
    plt.axvline(x=vline, color='#E2231A', linestyle='--', label=f'{mean_median.title()} ({vline:.1f})')

    # Add title and labels
    plt.title(title)
    plt.xlabel(x_label)
    plt.xticks(x_ticks)
    plt.ylabel('Frequency')
    plt.legend()
    plt.tight_layout()
    plt.show();

___
## Leads

### Data

Lead data comes from two distinct sources. To ensure integrity and usability, I merged the datasets into a unified DataFrame, removing duplicate columns and preserving unique information. Additionally, I anonymized the data to maintain confidentiality and protect proprietary details.

In [ ]:
# Inspect Contents
leads= pd.read_csv(leads_location)
leads = standardize_columns(leads)
leads.head()

In [ ]:
# See shape of df
leads.shape

In [ ]:
# Replace 0 for NaN in zipcodes
leads['zip_code'].replace(0, np.NaN, inplace=True)

In [ ]:
# View Data Types
leads.info()

In [ ]:
# Count nulls -- Note: some still exist as non 'null' values
leads.isna().sum()

In [ ]:
# Not very helpful for zipcodes
leads.describe()

In [ ]:
# Transform created_date to datetime and make index
leads['created_date'] = pd.to_datetime(leads['created_date'])
leads.sort_values(by='created_date', ascending=True, inplace=True)
leads.set_index('created_date', inplace=True)

In [ ]:
# Calculate date range of data
leads_min_date = leads.index.min().strftime('%Y-%m-%d')
leads_max_date = leads.index.max().strftime('%Y-%m-%d')
print(f'The data includes leads from {leads_min_date} to {leads_max_date}.')

#### Data Visualization

In [ ]:
plot_categorical(leads)

Here we see:
* Most leads are 'open' which gives rise to the opportunity of advancement in the pipeline.
* Most leads have not been converted, not uncommon in sales.
* Mostly balanced split between male and female students. Other may account for non-binary or null information.
* Most students are now college level.
* Most leads have no specified lead source. Most identified leads come from the internet with location being the second most dominant type.

In [ ]:
plot_numeric(leads)

This graph does not tell us much because the zip codes are encoded

#### Data Summary

There are 1,533 leads from Aug 2011 - Dec. 2024 measured on 7 features. These include:
* created_date - When the lead first entered the system.
* zip_code - Geographic location; masked for anonyminity. 0 Represents null values.
* lead_status - Where they are at in the pipelien; 36 untracked leads.
* conversion_status - Whether or not they enrolled in service; no nulls.
* student_gender - Most nulls (1,159), indicating this data is not collected consistently upon initial lead contact.
* grade - 727 null values; 'college' and 'k' are forcing string type. Convert to integers.
* lead_source - Nulls represented as 'Not Specified'

### Analysis

#### Lead Volume Over Time

In [ ]:
# Group leads by month
monthly_leads = leads.resample('M').agg(
    converted_leads=('conversion_status', lambda x: (x == 'Converted').sum()),      # Use conversion_status because it has no nulls
    lead_count=('conversion_status', 'count')     
)
monthly_leads

In [ ]:
plot_x_over_time(
    x=monthly_leads.index,
    y=monthly_leads['lead_count'],
    title='Average Leads Over Time',
    y_label='Average Leads',
    line_label='Monthly Leads'
)

Here we see that leads prior to 2017 were extremely low on a monthly basis. Let's focus on more recent years where the data is more consistent.

In [ ]:
# Filter data starting in 2017.
# Data should include last full month. End data Nov 2024.
filtered_monthly_leads = monthly_leads[(monthly_leads.index >= '2017-01-01') & (monthly_leads.index < '2024-12-01')]

In [ ]:
plot_x_over_time(
    x=filtered_monthly_leads.index,
    y=filtered_monthly_leads['lead_count'],
    title='Average Leads Over Time',
    y_label='Average Leads',
    line_label='Monthly Leads'
)
plt.savefig('images/location_1/test.png')

Here we see average leads per month is 16.

#### Lead Volume per Month

In [ ]:
# Create 'month' column and set equal to 3 letter abbreviation
filtered_monthly_leads['month'] = filtered_monthly_leads.index.month_name().str[:3]

# Group by month and calculate the mean for each
monthly_mean = filtered_monthly_leads.groupby('month').mean()

monthly_mean

In [ ]:
plot_x_per_month(
    y=monthly_mean['lead_count'],
    title='Average Leads per Month',
    y_label='Average Leads'
)

Here we see that lead volume is:
* Lowest in June (10) & July (8) -- Corresponding with summer
* Highest in September (26) and October (31) -- Roughly corresponding with the end of the 1st quarter

##### Statistical Testing

Because I am comparing multiple groups I will conduct an ANOVA test.
* H0: There is not a significant difference in lead volume for different months of the year.
* H1: There is a significant difference in lead volume for different months of the year.
* Alpha = 0.05

In [ ]:
# Run ANOVA Test
perform_anova(filtered_monthly_leads, 'month', 'lead_count', 0.05)

Because the above result was significant I will follow up with Tukey's HSD to find which months have a significant difference.
* H0: The mean lead volume of month A and month B are not significantly different.
* H1: The mean lead volume of month A and month B are significantly different.
* Alpha = 0.05

In [ ]:
# Follow up with Tukey's HSD
lead_count_tukey = perform_tukeys_hsd(filtered_monthly_leads, 'month', 'lead_count', 0.05)
lead_count_tukey

* October has significantly more leads than April, May, June, July, and December.
* September has significantly more leads than July. 
* All other months do not show a significant difference

#### Number of Leads Converted Over Time

In [ ]:
plot_x_over_time(
    x=filtered_monthly_leads.index,
    y=filtered_monthly_leads['converted_leads'],
    title='Leads Converted Over Time',
    y_label='Average Leads Converted',
    line_label='Monthly Leads Converted'
)

Here we see average number of leads converted per month is 5.

#### Number of Leads Converted per Month

In [ ]:
plot_x_per_month(
    y=monthly_mean['converted_leads'],
    title='Average Leads Converted per Month',
    y_label='Average Leads Converted'
)

Here we see:
* Number of leads converted is lowest in July (summer)
* Number of leads converted is highest in September (back to school)

##### Statistical Testing

Because I am comparing multiple groups I will conduct an ANOVA test.
* H0: There is not a significant difference in number of converted leads for different months of the year.
* H1: There is a significant difference in number of converted leads for different months of the year.
* Alpha = 0.05

In [ ]:
# Run ANOVA Test
perform_anova(filtered_monthly_leads, 'month', 'converted_leads', 0.05)

Because the above result was significant I will follow up with Tukey's HSD to find which months have a significant difference.
* H0: The mean number of converted leads of month A and month B are not significantly different.
* H1: The mean number of converted leads of month A and month B are significantly different.
* Alpha = 0.05

In [ ]:
# Follow up with Tukey's HSD
lead_count_tukey = perform_tukeys_hsd(filtered_monthly_leads, 'month', 'converted_leads', 0.05)
lead_count_tukey

While the ANOVA test showed a statistically significant result, the difference in means between any two months did not meet the threshold of Tukey's HSD to qualify as a statistically significant result. This suggests that while there may be variation in lead conversions throughout the year, the difference in means between any two months is not large enough to be statistically significant.

#### Percent Lead Conversion Over Time

In [ ]:
plot_x_over_time(
    x=filtered_monthly_leads.index,
    y=filtered_monthly_leads['converted_leads'] / filtered_monthly_leads['lead_count'] * 100,
    title='Percent Lead Conversion Over Time',
    y_label='Lead Conversion (%)',
    line_label='Percent Monthly Lead Conversion'
)

Here we see average lead conversion is 33%.

#### Percent Lead Conversion per Month

In [ ]:
plot_x_per_month(
    y=monthly_mean['converted_leads'] / monthly_mean['lead_count'] * 100,
    title='Lead Conversion by Month',
    y_label='Lead Conversion (%)'
)

Here we see that percent lead conversion is:
* Lowest in October
* Highest in June

#### Grade

Grade is updated every year to keep track of students over time. We'll have to calculate the grade at time of lead creation to make this useful. Because 'college' (13) is a fixed value, students with that label will be excluded from this calculation.

`Note`: This will inherently introduce bias by omitting students in grades nearest to that label.

In [ ]:
# Create map to replace 'College' and 'K'
grade_map = {
    'College': 13,
    'K': 0
}

In [ ]:
# Apply map to 'grade' column
leads['grade'] = leads['grade'].replace(grade_map)

In [ ]:
# Copy Leads df and isolate those where Grade is not 13 or NaN
lead_student_grades = leads.copy()
lead_student_grades = lead_student_grades.loc[(lead_student_grades['grade'] != 13) & (lead_student_grades['grade'].notna())]

In [ ]:
# Calculate elapsed time
lead_student_grades['elapsed_years'] = (pd.to_datetime('now') - lead_student_grades.index)

# Convert to years
lead_student_grades['elapsed_years'] = round(lead_student_grades['elapsed_years'].dt.total_seconds() / (60 * 60 * 24 * 365.25), 0)
lead_student_grades.head()

In [ ]:
lead_student_grades['grade'] = lead_student_grades['grade'].astype(int)
lead_student_grades['grade_when_created'] = lead_student_grades['grade'] - lead_student_grades['elapsed_years']
lead_student_grades.head()

In [ ]:
lead_student_grades['grade_when_created'].value_counts()

In [ ]:
# Data to plot
x = lead_student_grades['grade_when_created'].value_counts().index
y = lead_student_grades['grade_when_created'].value_counts().values

# Plot
plt.bar(x, y, color='skyblue')

# Add title and labels
plt.title('Student Grade at Lead Creation')
plt.xlabel('Grade')
plt.xticks(range(int(x.min()), int(x.max()) + 1))
plt.ylabel('Number of Leads')
plt.show();

Here we see that some inquiries happen prior to Kindergarten, while most occur in 5th grade.

#### Lead Source

In [ ]:
# Isolate Converted Leads
converted_leads = leads.loc[leads['conversion_status'] == 'Converted']

x = converted_leads['lead_source'].value_counts().index
y = converted_leads['lead_source'].value_counts().values

plt.barh(x, y);

Here we see:
* All converted leads show lead source as null.

`Note`: After investigating the data source I found that Lead Source is tracked upon conversion but is limited to a small window of accessibility. For that reason I did not import the data to analyze further.

#### Leads by Zip Code

In [ ]:
# Top 5 Zip Codes by Lead Volume
top_zip_codes_count = leads['zip_code'].value_counts().nlargest(5)

# Plot
top_zip_codes_count.plot(kind='bar', color='skyblue')

# Add title and labels
plt.title('Top 5 Zip Codes by Lead Count')
plt.xlabel('Zip Code')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show();

Here we see that most leads come from the following zip codes:
* 1
* 5
* 6
* 15
* 4

In [ ]:
# Top 5 Zip Codes by Conversion
top_zip_codes_converted = converted_leads['zip_code'].value_counts().nlargest(5)

# Plot
top_zip_codes_converted.plot(kind='bar', color='skyblue')

# Add title and labels
plt.title('Top 5 Zip Codes by Conversion')
plt.xlabel('Zip Code')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show();

Here we see that most enrollments come from the following zip codes:
* 1
* 5
* 6
* 15
* 4

`Note`: The count for converted leads is almost identical to the count for all Leads. This indicates that this information is likely captured upon enrollment, not during initial lead contact.

### Recommendations

* Re-engage with ‘open’ leads to push them further in the pipeline.
* Demonstrate long term value to unconverted leads. Incentivize them to come in.
* With lead interest highest  at the end of the first quarter, focus on enrollment conversion.
    * Despite highest leads in October that is month of lowest percent conversion.
* With lead interest lowest over the summer, focus on retention strategies.
    * Although June is one of the months with the lowest lead volume, conversion percentage is highest.
* Ensure marketing strategy is tailored to target demographic:
    * Parents of late-elementary school students (grades 3-5).
    * Residents in zip codes 1, 5, and 6.

___
## Accounts

### Data

Account data comes from two distinct sources. To ensure integrity and usability, I merged the datasets into a unified DataFrame, removing duplicate columns and preserving unique information. Additionally, I anonymized the data to maintain confidentiality and protect proprietary details.

In [ ]:
# Inspect contents
accounts = pd.read_csv(accounts_location)
accounts = standardize_columns(accounts)
accounts.head()

In [ ]:
# See shape of df
accounts.shape

In [ ]:
# View Data Types
accounts.info()

In [ ]:
# Check for nulls -- Thankfully, NONE!
accounts.isna().sum()

In [ ]:
# Get stats for numeric columns
accounts.describe()

In [ ]:
# See membership types
accounts['membership_type'].value_counts()

In [ ]:
# 'total_sessions' is not a cumulative count
accounts['total_sessions'].value_counts()

#### Data Cleaning & Feature Engineering

* grade - Current grade; 'college' and 'k' are forcing string type. Convert to integers.
* membership_type - Remove 'credits' and consolidate remaining
* primary_enrollment_start - Convert to datetime
* primary_enrollment_end - Convert to datetime
* enrollment_contract_length - Remove unit from entry. Convert to integers.
* enrollment_length_of_stay - Remove unit from entry. Convert to integers.

In [ ]:
# Replace 'college' with 13 and 'k' with 0
accounts['grade'] = accounts['grade'].replace(grade_map)

In [ ]:
# Remove 'credit' type enrollments from data frame
accounts = accounts[accounts['membership_type'] != 'Credits (Monthly Sessions)']

In [ ]:
# Convert enrollment dates to date time
accounts['primary_enrollment_start'] = pd.to_datetime(accounts['primary_enrollment_start'])
accounts['primary_enrollment_end'] = pd.to_datetime(accounts['primary_enrollment_end'])

# Set index to enrollment start date and sort by date
accounts.set_index('primary_enrollment_start', inplace=True)
accounts.sort_index(ascending=True, inplace=True)

In [ ]:
# Strip enrollment_contract_length & enrollment_length_of_stay of the word 'months'
accounts['enrollment_contract_length'] = accounts['enrollment_contract_length'].str.split(' ').str[0]
accounts['enrollment_length_of_stay'] = accounts['enrollment_length_of_stay'].str.split(' ').str[0]

# Convert to float
accounts['enrollment_length_of_stay'] = pd.to_numeric(accounts['enrollment_length_of_stay'], errors='coerce')
accounts['enrollment_contract_length'] = pd.to_numeric(accounts['enrollment_contract_length'], errors='coerce')

In [ ]:
# Calculate date range of data
accounts_min_date = accounts.index.min().strftime('%Y-%m-%d')
accounts_max_date = accounts.index.max().strftime('%Y-%m-%d')
print(f'The data includes accounts from {accounts_min_date} to {accounts_max_date}.')

#### Data Visualization

In [ ]:
# Plot categorical columns
plot_categorical(accounts)

Here we see:
* Most students are now college level.
* Enrollment status is highly skewed toward Inactive.
* There are many membership types. Most are near 0 counts with the majority being of the 'Flexible' type.

In [ ]:
plot_numeric(accounts)

Here we see:
* Most contracts are less than one year.
* Enrollment length more evenly distributed with mean near 1 year.
* Total sessions primarily less than 20.
* Hardly any accounts referred.

#### Data Summary

There are 653 accounts from June 2013 - Dec. 2024 measured on 10 features. These include:
* grade - Current grade
* status - Enrolled or inactive
* membership_type - Type of enrollment
* primary_enrollment_start - Enrollment start date
* primary_enrollment_end - Enrollment end date
* enrollment_contract_length - Contract length at signup.
* enrollment_length_of_stay - Length of attendance.
* total_sessions - Allowable sessions based on enrollment type, NOT total attended.
* referral_account - Flag if the account was referred by another cusomter (yes = 1, no = 0)

### Analysis

#### Enrollments Over Time

In [ ]:
# Group enrollments by month
monthly_accounts = accounts['membership_type'].resample('M').count()    # Use membership_type because it has no nulls
monthly_accounts.index = pd.to_datetime(monthly_accounts.index)
monthly_accounts

In [ ]:
plot_x_over_time(
    x=monthly_accounts.index,
    y=monthly_accounts.values,
    title='Enrollments Over Time',
    y_label='Number of Enrollments',
    line_label='Enrollments'
)

Here we see the same issue as before. I'll filter the dataset in the same way.

In [ ]:
# Filter date >= 2017-01-01 and group by month
filtered_monthly_accounts_start = monthly_accounts[(monthly_accounts.index >= '2017-01-01') & (monthly_accounts.index < '2024-12-01')]

plot_x_over_time(
    x=filtered_monthly_accounts_start.index,
    y=filtered_monthly_accounts_start.values,
    title='Enrollments Over Time',
    y_label='Number of Enrollments',
    line_label='Enrollments'
)

Here we see that the monthly enrollment average is 7.

`Note`: This data includes students that re-enroll and siblings recorded under the same lead account so direct comparison to lead chart above will not match overall Lead Conversion.

#### Enrollments per Month

In [ ]:
# Create a 'month' column and set equal to 3 letter abbreviation
filtered_monthly_accounts_start = filtered_monthly_accounts_start.to_frame()
filtered_monthly_accounts_start['month'] = filtered_monthly_accounts_start.index.month_name().str[:3]
filtered_monthly_accounts_start.rename(columns={'membership_type': 'enrollments'}, inplace=True)

# Group by month and calculate the mean for each
enrollment_mean = filtered_monthly_accounts_start.groupby('month').mean()
enrollment_mean

In [ ]:
plot_x_per_month(
    y=enrollment_mean['enrollments'],
    title='Average Enrollments per Month',
    y_label='Average Enrollments'
)

Here we see enrollments are:
* Lowest in April (just prior to finals)
* Highest in September (back to school)

#### Enrollments by Membership Type

In [ ]:
# Check counts of membership types
accounts['membership_type'].value_counts()

In [ ]:
# Create map to consolidate membership types
def map_program_type(program_type):
    if 'Flexible' in program_type:
        return 'Flexible'
    elif 'Monthly' in program_type:
        return 'Monthly Sessions'
    elif 'Private' in program_type:
        return 'Private Sessions'
    elif 'Sessions' in program_type:
        return 'Session Packages'
    else:
        return 'Other'

In [ ]:
# Apply mapping to membership_type
accounts['membership_type'] = accounts['membership_type'].apply(map_program_type)
membership_type_counts = accounts['membership_type'].value_counts()
membership_type_counts

In [ ]:
# Percentage of recurring enrollment types
percent_recurring_enrollments = (membership_type_counts['Flexible'] + membership_type_counts['Monthly Sessions']) / membership_type_counts.sum() * 100
print(f'The percentage of enrollments that choose a recurring program is: {percent_recurring_enrollments:.0f}%')

In [ ]:
# Data to plot
x = accounts['membership_type'].value_counts().index
y = accounts['membership_type'].value_counts().values

# Plot
plt.barh(x, y, color='skyblue')

# Add title and labels
plt.title('Membership Types and Counts');
plt.xlabel('Count')
plt.ylabel('Membership Type')
plt.show();

Here we see that the dominant enrollment type is the 'Flexible' option.

In [ ]:
# Isolate memberships by subtype ('total_sessions' allowed)
monthly_membership_subtypes = accounts.copy()
monthly_sessions = monthly_membership_subtypes['membership_type'] == 'Monthly Sessions'
monthly_membership_subtypes.loc[monthly_sessions, 'membership_type'] = monthly_membership_subtypes.loc[monthly_sessions, 'total_sessions'].astype(str) + 'x'

In [ ]:
# Data to plot
x = monthly_membership_subtypes['membership_type'].value_counts().index
y = monthly_membership_subtypes['membership_type'].value_counts().values

# Plot
plt.barh(x, y, color='skyblue')

# Add title and labels
plt.title('Membership Types and Counts');
plt.xlabel('Count')
plt.ylabel('Membership Type')
plt.show();

Here we see that of the monthly membership options, the 8x program subtype is most common.

#### Enrollments by Grade Range

In [ ]:
# Copy Accounts df and isolate those where Grade is not NaN or 13
account_student_grades = accounts.copy()
account_student_grades = account_student_grades.loc[(account_student_grades['grade'] != 13) & (account_student_grades['grade'].notna())]

# Calculate elapsed time
account_student_grades['elapsed_years'] = (pd.to_datetime('now') - account_student_grades.index)

# Convert to years
account_student_grades['elapsed_years'] = round(account_student_grades['elapsed_years'].dt.total_seconds() / (60 * 60 * 24 * 365.25), 0)
account_student_grades.head()

account_student_grades['grade'] = account_student_grades['grade'].astype(int)
account_student_grades['grade_when_created'] = account_student_grades['grade'] - account_student_grades['elapsed_years']
account_student_grades.head()

In [ ]:
# Data to plot
x = account_student_grades['grade_when_created'].value_counts().index
y = account_student_grades['grade_when_created'].value_counts().values

# Plot
plt.bar(x, y, color='skyblue')

# Add title and labels
plt.title('Student Grade at Enrollment')
plt.xlabel('Grade')
plt.xticks(range(int(x.min()), int(x.max()) + 1))
plt.ylabel('Number of Enrollments')
plt.show();

Here we see that:
* Inquiries start coming in 3 years prior to kindergarten (0).
* Student enrollment is highest for 5th graders.
* Data is normally distributed.

In [ ]:
# Create new column for grade range
account_student_grades['grade_range'] = account_student_grades['grade_when_created'].apply(assign_grade_range)

In [ ]:
account_student_grades.head()

In [ ]:
account_student_grades['grade_range'].value_counts()

In [ ]:
# Group by grade_range and membership_type
membership_by_grade_range = account_student_grades.groupby(['grade_range', 'membership_type']).size()
membership_by_grade_range

In [ ]:
# Create pivot table
membership_pivot = membership_by_grade_range.unstack(fill_value=0)
membership_percent = membership_pivot.div(membership_pivot.sum(axis=1), axis=0)  # Row-wise normalization

In [ ]:
# Specify display order
grade_order = ['Elementary', 'Middle', 'High']
membership_percent.index = pd.CategoricalIndex(membership_percent.index, categories=grade_order, ordered=True)
membership_percent = membership_percent.sort_index() * 100

# Plot side-by-side percentage bars
membership_percent.plot(
    figsize=(10, 6),
    kind='bar',
    stacked=False,
    color=['skyblue', 'green', 'purple', '#E2231A', 'orange']
)

# Add title and labels
plt.title('Percentage of Membership Types by Grade Range')
plt.xlabel('Grade Range')
plt.xticks(rotation=0)
plt.ylabel('Percentage')
plt.legend(title='Membership Type', bbox_to_anchor=(1.05, 1), loc='upper left')  # Move legend outside the plot
plt.tight_layout()
plt.show();

Here we see:
* Elementar school students predominantly enroll in the flexible membership option.
* Middle school students still enroll in the flexible option most often but with monthly sessions a closer second option.
* High school students enroll in near-equal numbers for the flexible and monthly options with a large jump in private session enrollments.

#### Cancelations Over Time

Enrollment End Dates by Month

In [ ]:
# Copy accounts df and set primary_enrollment_end as the index
accounts_end = accounts.copy()
accounts_end.set_index('primary_enrollment_end', inplace=True)

# Filter date >= 2017-01-01 and group by month
filtered_accounts_end = accounts_end[(accounts_end.index >= '2017-01-01') & (accounts_end.index < '2024-12-01')]
filtered_monthly_accounts_end = filtered_accounts_end['membership_type'].resample('M').count()    # Use membership_type because it has no nulls
filtered_monthly_accounts_end.index = pd.to_datetime(filtered_monthly_accounts_end.index)
filtered_monthly_accounts_end.head()

In [ ]:
plot_x_over_time(
    x=filtered_monthly_accounts_end.index,
    y=filtered_monthly_accounts_end.values,
    title='Cancelations Over Time',
    y_label='Number of Cancelations',
    line_label='Cancelations'
)

Here we see that average monthly cancelations is 6.

#### Cancelations per Month

In [ ]:
# Create a 'month' column and set equal to 3 letter abbreviation
filtered_monthly_accounts_end = filtered_monthly_accounts_end.to_frame()
filtered_monthly_accounts_end['month'] = filtered_monthly_accounts_end.index.month_name().str[:3]
filtered_monthly_accounts_end.rename(columns={'membership_type': 'cancelations'}, inplace=True)

# Group by month and calculate the mean for each
cancelation_mean = filtered_monthly_accounts_end.groupby('month').mean()
cancelation_mean

It's a little peculiar that August is so high given that it is the start of the school year. Let's check for outliers.

In [ ]:
# August has severl high cancelation years so we'll leave it as is
filtered_monthly_accounts_end.loc[filtered_monthly_accounts_end['month'] == 'Aug']

In [ ]:
plot_x_per_month(
    y=cancelation_mean['cancelations'],
    title='Average Cancelations per Month',
    y_label='Average Canccelations'
)

Here we see that cancelations are:
* Lowest in November (middle of second quarter).
* Highest in August (start of new school year; "we'll see how this year goes").

#### Net Monthly Change in Enrollments

In [ ]:
# Check structure
enrollment_mean

In [ ]:
# Check structure
cancelation_mean

In [ ]:
# Calculate net monthly change in enrollments
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

net_monthly_change = enrollment_mean['enrollments'].reindex(month_order) - cancelation_mean['cancelations'].reindex(month_order)
net_monthly_change

In [ ]:
# Plot
plt.figure(figsize=(10, 6))
colors = ['#E2231A' if value < 0 else 'skyblue' for value in net_monthly_change]
net_monthly_change.plot(kind='bar', color=colors, legend=False)
plt.axhline(0, color='black')  # Add a reference line at 0

# Add title and labels
plt.title('Average Net Monthly Change')
plt.xlabel('Month')
plt.ylabel('Average Change in Enrollments')
plt.xticks(rotation=0)
plt.tight_layout();

Here we see that:
* There are 6 months of the year that have net negative enrollment (more cancelations than enrollments) with August being the worst month.
* There are 6 months of the year that have net positive enrollment (more enrollments than cancelations) with September being the best month.

#### Enrollment Completion

In [ ]:
# Percentage of families that cancel before they complete their agreement
early_cancel = accounts.loc[accounts['enrollment_length_of_stay'] < accounts['enrollment_contract_length']]
early_cancel_percentage = len(early_cancel) / len(accounts) * 100
print(f'The percentage of families that complete their entire enrollment agreement is {100 - early_cancel_percentage:.0f}%')
print(f'The percentage of families that cancel their program before the end of their agreement is: {early_cancel_percentage:.0f}%')

In [ ]:
mean_length_of_stay = accounts['enrollment_length_of_stay'].mean()
print(f'On average students stay enrolled for {mean_length_of_stay:.1f} months per enrollment agreeement.')

In [ ]:
accounts['enrollment_length_of_stay'].loc[accounts['enrollment_length_of_stay'] < 0]

In [ ]:
plot_histogram

In [ ]:
# Visualize
# Filtered less than 40 to exclude outliers and enhance visual
plot_histogram(
    data=accounts['enrollment_length_of_stay'].loc[(accounts['enrollment_length_of_stay'] >= 0) & (accounts['enrollment_length_of_stay'] < 40)],
    title='Distribution of Lenth of Enrollment',
    x_label='Months',
    x_ticks_interval=5
)

Here we see that length of enrollment is skewed right. Median length of enrollment is 4.6 months.

### Recommendations

* Re-engage ‘inactive’ accounts for re-enrollment.
* Create a referral program to simultaneously increase revenue and reputation.
* Tailor program offering to grade range.
* Upsell current customers by offering private sessions at key times of the year.
* Increase retention through active communication and progress reporting, especially leading up to August.
* Incentivize longer commitments to increase average length of stay.

___
## Student Progress per Assessment Level

### Data

Assessment data comes from a single source. I anonymized the data to maintain confidentiality and protect proprietary details.

In [ ]:
# Inspect contents
assessment_progress = pd.read_csv(assessment_progress_location)
assessment_progress = standardize_columns(assessment_progress)
assessment_progress.head()

In [ ]:
# Set index to pre-assessment date and sort by date
assessment_progress['pre_assess_date'] = pd.to_datetime(assessment_progress['pre_assess_date'])
assessment_progress.sort_values(by='pre_assess_date', ascending=True, inplace=True)
assessment_progress.set_index('pre_assess_date', inplace=True)

In [ ]:
assessment_progress.shape

In [ ]:
# Check data types
assessment_progress.info()

In [ ]:
# Check for nulls
assessment_progress.isna().sum()

In [ ]:
# Get stats for numeric columns
assessment_progress.describe()

#### Data Cleaning & Feature Engineering

* grade_on_post_dt. - Convert to int.
* assessment_level - Can be used to calculate disparity between grade and perforance. Convert to int.
* post_assess_date - Convert to datetime.

Will need to look at nulls:
* 15 in grade_on_post_dt.
* 18 in several other columns (same data?)

In [ ]:
# Use grade_map to replace 'K' and 'College'
assessment_progress['grade_on_post_dt.'] = assessment_progress['grade_on_post_dt.'].replace(grade_map)

# Make post_assess_date datetime
assessment_progress['post_assess_date'] = pd.to_datetime(assessment_progress['post_assess_date'])

In [ ]:
# See all assessments with Null grade_on_post_dt. to look for patterns
assessment_progress.loc[assessment_progress['grade_on_post_dt.'].isna() == True]

In [ ]:
# Find the median post grade for each assessment level
median_post_grade_per_assessment_level = assessment_progress.groupby(['assessment_level'])['grade_on_post_dt.'].median()

# Fill nulls with asessment-level median grade
assessment_progress['grade_on_post_dt.'] = assessment_progress['grade_on_post_dt.'].fillna(
    assessment_progress['assessment_level'].map(median_post_grade_per_assessment_level)
)

In [ ]:
# NF type assessment gets assigned a mastery level, not a numeric score. 
# Because structure and depth of learning plan differ from other types of assessments I'll drop these.
assessment_progress.loc[assessment_progress['pre_assess_score'].isna() == True]

In [ ]:
# Calculate percent of assessments that are NF
percentage_nf_assessments = len(assessment_progress.loc[assessment_progress['pre_assess_score'].isna() == True]) / len(assessment_progress) * 100
print(f'{percentage_nf_assessments:.0f}% of all assessments are NF.')

In [ ]:
# Drop NF-type assessments
assessment_progress = assessment_progress.dropna()

In [ ]:
# Convert grade_on_post_dt. and assessment_level to numeric columns
assessment_progress['grade_on_post_dt.'] = pd.to_numeric(assessment_progress['grade_on_post_dt.'])
assessment_progress['assessment_level'] = pd.to_numeric(assessment_progress['assessment_level'])

In [ ]:
# Calculate performance relative to grade level
assessment_progress['relative_performance'] = assessment_progress['assessment_level'] - assessment_progress['grade_on_post_dt.']

#### Data Visualization

In [ ]:
plot_numeric(assessment_progress)

Here we see that:
* Grade at time of post assessment peaks around 4th grade.
* Levels 3-4 are the most common assessment types.
* Most students take the post-assessment 1-2 times.
* Pre assessment scores aretypically in the range of 30-70%.
* Post assessment scores have a strong downward skew (higher scores increasingly more frequent).
* Number of PKs mastered per level is primarily less than 30.
* Score improvement is around 25-35%.
* Improvement per session is concentrated in a small range.
* Improvemnet per month is concentrated in a small range.
* Attendance has strong positive skew.
* Relative performance is mostly 2 years under grade level to at grade level.

#### Data Summary

Assessment data includes all assessments for students where at least one post assessment has been attempted. There are 356 assessments matching this criteria from 2000-01-01 to 2024-12-09. Each assessment is measured across 12 features:
* grade_on_post_dt. - Grade of student at time of post assessment.
* assessment_level - Grade level equivalent of student performance. NF assessments (5% of total) were dropped.
* #_post_attempts - Number of attempts on post-assessment (>= 80 moves to next assessment level).
* pre_assess_score - Range between 8-100% with mean and median near 45%.
* post_assess_score - Range between 11-100% with mean and median near 80%.
* post_assess_date - Date of post-assessment.
* #_pks_mastered - Lessons mastered between pre and post dates. Mean is 14, median is 11.
* score_improvement - Change in score from pre-assessment to post-assessment.
* improvement/session - Improvement divided by attendance for assessment level. 
* improvement/month - Improvement divided by time between pre and post assessment. Roughly +6-7% per month.
* attendance_count_in_center - Number of sessions in assessment period. Positive skew with mean at 33 and median at 25.
* relative_performance - Comparison of student assessment level to grade level. A negative value indicates they are below grade level. 

### Analysis

#### Relative Student Performance

In [ ]:
# Median test-taker relative_performance
median_relative_performance = assessment_progress['relative_performance'].median()
print(f'Students who have completed a pre-assessment and post-assessment for the same level are, on approximately {median_relative_performance:.1f} grades from grade level.')

In [ ]:
# Visualize
plot_histogram(
    data=assessment_progress['relative_performance'],
    title='Distribution of Relative Student Performance',
    x_label='Performance Relative to Grade Level',
)

Here we see that relative student performance:
* Ranges from 8 years below grade level to 3 years above grade level.
* Median 1.0 year below grade level.

`NOTE`: This includes data for students that have completed multiple levels and excludes students that have yet to take a post assessment. Therefore students entering the program are likely below the median.

#### Performance per Post-Assessment Attempt

In [ ]:
# Loop calculates average change in assessment score based on number of attempts

# Initialize
cumulative_performance = 0
performance_per_attempt = []

for attempt in range(1, assessment_progress['#_post_attempts'].max() + 1):

    # Calculate current iteration performance and take away effects of previous iterations
    current_performance = assessment_progress.loc[assessment_progress['#_post_attempts'] == attempt]['score_improvement'].mean() - cumulative_performance
    print(f"Average performance after {attempt} attempt(s) is: {(current_performance) * 100:.1f}%")

    # Add to cumulative performance for deduction in later iterations
    cumulative_performance += current_performance
    performance_per_attempt.append((attempt, current_performance * 100))

In [ ]:
performance_per_attempt

In [ ]:
# Data to plot
performance_df = pd.DataFrame(performance_per_attempt, columns=['attempt', 'performance'])
x = performance_df['attempt']
y = performance_df['performance']

colors = ['#E2231A' if value < 0 else 'skyblue' for value in performance_df['performance']]

# Plot
plt.figure(figsize=(10, 6))
plt.bar(x, y, color=colors)
plt.axhline(0, color='black')  # Add a reference line at 0

# Add title and labels
plt.title('Performance Change by Post-Assessment Attempt')
plt.xlabel('Attempt Number')
plt.ylabel('Average Change (%)')
plt.tight_layout()
plt.show();

Here we see a strong initial increase in scoring from pre-assessment to post-assessment followed by moderate growth on subsequent attempts before finally declining. The decrease in performance over time may be caused by several factors:
* There is less content to master as score rises.
* The remaining content is especially challenging for the student.
* The student may feel burntout after several iterations of the same content.

#### Student Advancement

Generally, students are promoted to the next level when they get at least 80% on their post assessment. Let's take a closer look at the trends that allow a student to move from one level to the next.

In [ ]:
# Isolate rows where post_assess_score >= 80%
next_level_assessments = assessment_progress.loc[assessment_progress['post_assess_score'] >= 0.8]
next_level_assessments.shape

In [ ]:
# Calculate elapsed time in months
next_level_assessments['elapsed_months'] = round((next_level_assessments['post_assess_date'] - next_level_assessments.index).dt.days / 30.44, 1)
next_level_assessments.head()

In [ ]:
next_level_assessments.describe()

* Average attempts is 1.5 to move from one level to the next.
* Students who complete a given level average 92% on their post-assessment.
* It takes roughly 15-17 PKs to reach mastery from initial assessment score near 55%.
* It takes roughly 25 (median) to 34 (mean) visits to progress from one level to the next.
* It takes roughly 4.5 (median) to 6 (mean) months to progress from one level to the next.

##### Post Assessment Attempts

In [ ]:
# Visualize
plot_histogram(
    data=next_level_assessments['#_post_attempts'],
    title='Distribution of Post Assessment Attempts',
    x_label='Post Assessment Attempts',
    mean_median='mean'
)

Here we see that most students move to the next level within 2 attempts.

##### PKs per Level

In [ ]:
# Visualize
plot_histogram(
    data=next_level_assessments['#_pks_mastered'],
    title='PKs per Completed Level',
    x_label='Number of PKs',
    bins=20,
    x_ticks_interval=10
)

Here we see that there is a strong positive skew for PKs required to complete each level. The median is 15.0.

##### Visits per Level

In [ ]:
# Visualize
# Filtered less than 180 to exclude outliers and enhance visual
plot_histogram(
    data=next_level_assessments['attendance_count_in_center'].loc[next_level_assessments['attendance_count_in_center'] < 180],
    title='Visits per Completed Level',
    x_label='Number of Visits',
    bins=20,
    x_ticks_interval=20
)

Here we see that the number of visits has an extremely large range with strong positive skew. The median number of visits for a student to move to the next level is 25.

##### Time per Level

In [ ]:
# Visualize
# Filtered less than 40 to exclude outliers and enhance visual
plot_histogram(
    data=next_level_assessments['elapsed_months'].loc[next_level_assessments['elapsed_months'] < 40],
    title='Months per Completed Level',
    x_label='Months',
    bins=30,
    x_ticks_interval=5,
    mean_median='mean'
)

Here we see that:
* There is a strong positive skew.
* Most students move to the next level within the first year.
* The median time is 4.6 months.

`Note`: The discrepancy between number of visits required (and therefore time) to complete a level is influenced by factors such as:
* Score on pre-assessment
* The portion of the sessions dedicated to PKs vs school work
* Consistent student attendance

### Recommendations

* Highlight ability to work with students of all ability levels
* Flag students that are failing to achieve a passing score after their second post-assessment to understand reasons why.
    * Target higher pre-assessment scores so students are more likely to pass post-assessment within two attempts.
* Monitor PK completion, checking for true mastery along the way.
* Use time-based metrics to set expectations upon enrollment and to communicate progress toward long term objectives over the course of their enrollment.
    * Visits per Level: 25-35
    * Time per Level: 4.6-6 months


___
## Results

### Leads

* Lead Volume
    * Cyclical pattern, except during COVID where it remained at depressed levels.
    * New monthly leads average 16.
    * New leads are highest in Sept. & Oct. (middle to end of the first quarter).
    * New leads are lowest during June & July (summer).

* Leads Converted
    * Cyclical pattern but with post-COVID years averaging lower than pre-COVID years.
    * Average monthly leads converted is 5.
    * Lead conversion is highest in Sept, and the first three months of the calendar year.
    * Lead conversion is lowest in Apr. & July (end of school year & summer respectively).

* Percent Lead Conversion
    * Cyclical pattern with poorest performance starting in 2023.
    * Average monthly conversion is 33%.
    * Highest conversion rate is in May & June (families more serious about getting summer help).
    * Lowest conversion is in Aug. & Oct. (opportunity zone!)

* Other Info
    * Leads are most common for elementary-aged students, specifically grades 2-5.
    * Leads who enroll in the program are most commonly from zip codes 1, 5, and 6.

### Accounts

* Average monthly enrollment is 7, indicating mild success with re-enrollments of same students and/or sibling enrollments.
* The dominant enrollment type is the Flexible option. Broken down by grade range we see:
    * Elementary: 68% enrolled in Flexible option with 22% enrolled in Monthly Sessions.
    * Middle: 54/37 split
    * High: 41/37 with a large increase in demand for private sessions (18)
* Student grade at enrollment shifts slightly upward to grades 4-6.
* Average monthly cancelations is 6.
    * Highest month is Aug. Possibly explained by families wanting to "see how this year goes" before committing to longer term option. (opportunity zone!)
    * Lowest month is Nov. (heading toward semester 1 finals).
* Net change in enrollments shows similar patterns (positive in Sept., negative in Aug.)
    * Average Net change per year is +3.
* Enrollment length is 4.6 (median) to 6.2 (mean) months.

### Student Progress per Assessment Level

* Relative student performance is 1.5 (mean) to 1.0 (median) BELOW grade level.
* Students progress most upon first post-assessment attempt at 27%.
    * Subsequent attempts plateau near 7-8% improvement before declining at the 5th attempt.
* Most students pass to the next level within 1-2 attempts.
* Per level metrics:
    * PKs completed is approximately 15.
    * Attendance is 25 sessions.
    * Time is 4.6 (median) to 6.0 (mean) months.

___
## Conclusions

This analysis leads to the following recommendations:

### Leads

* Re-engage with ‘open’ leads to push them further in the pipeline.
* Demonstrate long term value to unconverted leads, incentivizing them to come in for assessments if necessary.
* With lead interest highest  at the end of the first quarter, focus on enrollment conversion during that time.
    * Best opportunity is in October since it consistently has the most leads but also the lowest conversion rate.
* With lead interest lowest over the summer:
    * Focus marketing efforts to capture additional lead interest.
    * Focus on retention strategies for enrolled families.
* Ensure marketing strategy is tailored to target demographic:
    * Parents of late-elementary school students (grades 3-5).
    * Residents in zip codes 1, 5, and 6.

### Accounts

* Re-engage ‘inactive’ accounts for re-enrollment.
* Create a referral program to simultaneously increase revenue and reputation.
* Tailor program offering to grade range.
* Upsell current customers (especially high schoolers) by offering private sessions at key times of the year.
* Increase retention through active communication and progress reporting, especially leading up to August.
* Incentivize longer commitments to increase average length of stay.

### Student Progress per Assessment Level

* Highlight capability to work with students of all ability levels to build trust and credibility with new leads. 
* Flag students that are failing to achieve a passing score after their second post-assessment to understand reasons why.
    * Target higher pre-assessment scores so students are more likely to pass post-assessment within two attempts.
* Monitor PK completion, checking for true mastery along the way.
* Use time-based metrics to set expectations for interested families and to advise best program to match their objectives to their child's current perforamnce level. These should also be used to communicate progress toward long-term objectives over the course of their enrollment.
    * Visits per Level: 25-35
    * Time per Level: 4.6-6.0 months

___
## Next Steps

* Use long term and monthly understanding of lead volume and lead conversion/enrollment data to:
    * Make informed decisions about promotional offerings:
        * Incentives for new families to get started.
        * Discounts for continuous enrollment beyond average.
    * Provide appropriate staff training at critical times of the year:
        * Sales training leading up to October.
        * Retention training leading toward the summer.
    * Create structured and realistic expectations of management staff performance.
        * Enhance staff motivation by offering bonuses for exceptional conversion rates.
* Use demographic information to:
    * Create unique marketing campaigns based on location.
    * Tailor marketing efforts to parents of late-elementary and early-middle school students.
* Use student assesssment/enrollment information to:
    * Make program recommendations based on student profile.
    * Strategize on approaches to extend student enrollment.
    * Monitor enrolled students' progress and:
        * Intervene when they are not meeting goals.
        * Communicate with parents about current standing and confirm/reset expectations for the future.

___
## Opportunities for Improvement

* Clean duplicate leads and accounts on the backend so future reports are cleaner and more accurate.
* Collect lead information (zip code, student grade, etc.) consistently to allow more insights into customer demographic. This will enhance targeted marketing efforts.
    * If possible, record grade of student upon initial interest, at start of each enrollment, and over time to maintain an accurate representation of students' grades at each step in the pipeline.
* Ensure all private sessions are recorded as enrollments and not simlply billed to the account.
* Always do a post-assessment (progress check) to correctly capture and measure Assessment Progress rather than moving a student to the next level.